# GVB Data

## Imports <a name="imports"></a>

In [9]:
# For JSON imports
import json

#For DF, CSV, Excel
import pandas as pd

import re
import datetime

## Functions

### Constrruct DF
With number of arrivals and departures of each station on a single row. 

In [5]:
def stationData(arr_df, dep_df, stations):

    #Dict to temp save DF's in
    arr_dict = {}
    dep_dict = {}

    #for station in stations, construct a custom temp df
    for station in stations:
        temp_arr_df = arr_df[arr_df["AankomstHalteNaam"] == station]

        temp_arr_df = temp_arr_df.rename(index=str, columns={"AantalReizen": station + " Arrivals",
                                                        "UurgroepOmschrijving (van aankomst)": "Hour", "Datum": "Date"})
                                         
        temp_arr_df = temp_arr_df.groupby(["Date", "Hour"]).agg({station + " Arrivals": 'sum'}).reset_index()

        temp_dep_df = dep_df[dep_df["VertrekHalteNaam"] == station]
                                         
        temp_dep_df = temp_dep_df.rename(
            index=str, columns={"AantalReizen": station + " Departures", "UurgroepOmschrijving (van vertrek)": "Hour", 
                                "Datum": "Date"})

        temp_dep_df = temp_dep_df.groupby(["Date", "Hour"]).agg(
            {station + " Departures": 'sum'}).reset_index()

        arr_dict["{0}".format(station)] = temp_arr_df
        dep_dict["{0}".format(station)] = temp_dep_df

    for i in range(len(stations)-1):
        arr_dict[stations[i+1]] = pd.merge(arr_dict[stations[i]],
                                           arr_dict[stations[i+1]], on=["Date", "Hour"], how="outer")

        dep_dict[stations[i+1]] = pd.merge(dep_dict[stations[i]],
                                           dep_dict[stations[i+1]], on=["Date", "Hour"], how="outer")

    return pd.merge(arr_dict[stations[-1]], dep_dict[stations[-1]],
             on=["Date", "Hour"], how="outer")

### Convert Date String to Datetime

In [13]:
def TransformData(df, stations):

    #Variables
    date_format_1 = '%d/%m/%Y %H:%M:%S'
    date_format_2 = '%m/%d/%Y %H:%M:%S'
    
    #Fill NaN values with 0
    df = df.fillna(0.0)

    #Add column day numbers
    df["weekday"] = 99

    #Add whether column to indicate whether it is weekend
    df["is_weekend"] = 0

    #Dataframe to Dict
    df_dict = df.to_dict("index")

    #Loop over dict
    for k, v in df_dict.items():
        #Replace time string with time blok
        time_blok = v["Hour"][:5]
        time_blok = re.sub('[:]', '', time_blok)
        v["Hour"] = int(time_blok)

        if v["Hour"] == 0:
            v["Hour"] = 2400

        #Remove AM/PM from string
        v["Date"] = v["Date"][:-3]
        try:
            #Transform the date string to datatime.date object
            date = pd.Timestamp.strptime(v["Date"], date_format_1)
            #Transfrom date to weekday number
            v["weekday"] = date.weekday()
        except:
            #Transform the date string to datatime.date object
            date = pd.Timestamp.strptime(v["Date"], date_format_2)

            #Transfrom date to weekday number
            v["weekday"] = date.weekday()
        
        #Transform Date string to datetime object
        v["Date"] = date.date()

        #Check if weekday is in the weekend
        if date.weekday() == 5 or date.weekday() == 6:
            v["is_weekend"] = 1

        v["Date"] = date.date()

    return pd.DataFrame.from_dict(df_dict, orient="index")

## Import Data

In [10]:
arr_df = pd.read_csv("../../../Data_thesis/GVB/Datalab_Reis_Bestemming_Uur_20190402.csv", sep=";")
dep_df = pd.read_csv("../../../Data_thesis/GVB/Datalab_Reis_Herkomst_Uur_20190403.csv", sep=";")

## Run Functions

In [11]:
stations = ["Nieuwmarkt", "Nieuwezijds Kolk", "Dam", "Spui"]

In [16]:
full_df = stationData(arr_df, dep_df, stations)
full_df = TransformData(full_df, stations)

## Final DataFrame <a name="FinalDF"></a>
Make the model with the GVB data from *Dam* station

Variables final df:
- *Date*
    - Date the contents was gathered
    - Datetime object
- *Hour*
    - The hour the counts were gathered --> 100 means 01:00
    - int
- *weekday*
    - Day of the week in numbers --> 0: Monday, 1: Tuesday,..., 6: Sunday
    - int
- *is_weekend*
    - Whether is it is weekend or not
    - binary
- *Arrivals*
    - Number of passengers that travelled to the station
    - int
- *Departures*
    - Number of passengers that travelled from the station
    - int 

In [18]:
full_df.head()

,Date,Hour,Nieuwmarkt Arrivals,Nieuwezijds Kolk Arrivals,Dam Arrivals,Spui Arrivals,Nieuwmarkt Departures,Nieuwezijds Kolk Departures,Dam Departures,Spui Departures,weekday,is_weekend
0,2018-01-01,100,11.0,0.0,0.0,0.0,340.0,27.0,0.0,0.0,0,0
1,2018-01-01,200,48.0,0.0,21.0,0.0,175.0,130.0,39.0,0.0,0,0
2,2018-01-01,300,10.0,16.0,13.0,0.0,137.0,17.0,48.0,0.0,0,0
3,2018-01-01,400,16.0,0.0,0.0,0.0,48.0,46.0,34.0,0.0,0,0
4,2018-01-01,500,17.0,0.0,0.0,0.0,56.0,0.0,20.0,0.0,0,0


## DF to File <a name="CSV"></a>
Save the dataframe in a file, so that it can be imported for other uses. Later this will be probably be removed as the dataframe can simple function as input to another funtion. 

In [19]:
full_df.to_csv("../../../Data_thesis/Full_Datasets/GVBData.csv", index=False)